In [7]:
# import packages
import numpy as np
import pandas as pd
import os
import plotly.express as px
from IPython.display import display

%reload_ext autoreload
%autoreload 2

# Tell python where to look for modules. 
import sys
sys.path.append('../../../open-grid-emissions/src/')

# import local modules
import load_data
import validation
from filepaths import *

from column_checks import get_dtypes

# Specify the year for validation

In [8]:
year = 2020

## Compare Our Results to eGRID

In [18]:
# load our annual plant level data
annual_plant_results = pd.read_csv(
    f"{results_folder()}{year}/plant_data/annual/us_units/plant_data.csv",
    dtype=get_dtypes(),
)
plant_attributes = pd.read_csv(
    f"{outputs_folder()}{year}/plant_static_attributes_{year}.csv", dtype=get_dtypes()
)
annual_plant_results = annual_plant_results.merge(
    plant_attributes, how="left", on="plant_id_eia"
)

# add a egrid id
annual_plant_results = validation.add_egrid_plant_id(
    annual_plant_results, from_id="eia", to_id="egrid"
)

# Load the eGRID plant table
egrid_plant = validation.load_egrid_plant_file(year)

### Identify plants missing from our results that exist in eGRID


In [19]:
missing_from_calc, PLANTS_MISSING_FROM_CALCULATION = validation.identify_plants_missing_from_our_calculations(egrid_plant, annual_plant_results, year)
missing_from_calc

,ba_code,state,plant_id_egrid,plant_name,plant_primary_fuel,chp_flag,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,co2_mass_lb,co2_mass_lb_adjusted,fuel_consumed_mmbtu_ozone_season,fuel_data_source_annual,fuel_data_source_ozone,plant_id_eia
0,NaN,AK,60814,7-Mile Ridge Wind Project,WND,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,60814
1,NaN,AK,54452,Agrium Kenai Nitrogen Operations,NG,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54452
2,NaN,AK,57053,Alakanuk,DFO,Yes,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57053
18,NaN,AK,93,Blue Lake Hydro,WAT,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,93
49,NaN,AK,313,Green Lake,WAT,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12642,WAUW,WY,674,Pilot Butte,WAT,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,674
12644,WACM,WY,64847,Rail Tie Wind,WND,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,64847
12659,PACE,WY,62516,TB Flats,WND,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,62516
12662,PACE,WY,63972,Two Rivers Wind Facility,WND,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,63972


### Identify plants missing from eGRID that are in our calculations

In [20]:
egrid_plant[egrid_plant["plant_id_egrid"] == 1146]

,ba_code,state,plant_id_egrid,plant_name,plant_primary_fuel,chp_flag,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,co2_mass_lb,co2_mass_lb_adjusted,fuel_consumed_mmbtu_ozone_season,fuel_data_source_annual,fuel_data_source_ozone,plant_id_eia
3463,MISO,IA,1146,Harlan,DFO,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1146


In [21]:
missing_from_egrid, PLANTS_MISSING_FROM_EGRID = validation.identify_plants_missing_from_egrid(egrid_plant, annual_plant_results)

# how many of the plants missing from egrid have non-zero data
missing_from_egrid.loc[missing_from_egrid["fuel_consumed_mmbtu"] != 0, ["plant_id_eia","plant_name_eia","plant_primary_fuel","net_generation_mwh","fuel_consumed_for_electricity_mmbtu","fuel_consumed_mmbtu"]]

,plant_id_eia,plant_name_eia,plant_primary_fuel,net_generation_mwh,fuel_consumed_for_electricity_mmbtu,fuel_consumed_mmbtu
0,52106,Philadelphia Refinery,PRG,0.0,2380168.6,2380168.6


### Compare whether totals for plants with EPA IDs that differ from EIA IDs match

In [ ]:
annual_plant_results[
    annual_plant_results["plant_id_egrid"].duplicated(keep=False)
].groupby(["plant_id_egrid","plant_id_eia"]).sum()

In [ ]:
# identify where there is a single egrid plant id for multiple eia plant ids
double_ids = annual_plant_results[
    annual_plant_results["plant_id_egrid"].duplicated(keep=False)
]
double_ids = (
    double_ids.groupby("plant_id_egrid").sum()["net_generation_mwh"].reset_index()
)  # focus on net generation for now
# merge the egrid data
double_ids = double_ids.merge(
    egrid_plant[["plant_id_egrid", "net_generation_mwh"]],
    how="left",
    on="plant_id_egrid",
    suffixes=("_calc", "_egrid"),
)
double_ids["percent_diff"] = (
    (double_ids["net_generation_mwh_calc"] - double_ids["net_generation_mwh_egrid"])
    / double_ids["net_generation_mwh_egrid"]
).round(3)
double_ids


### Identify plants where our BA assignment does not match eGRID

In [ ]:
ba_code_match = egrid_plant.set_index("plant_id_eia")[["plant_name", "ba_code"]].merge(
    annual_plant_results.set_index("plant_id_eia")[["ba_code"]],
    how="inner",
    left_index=True,
    right_index=True,
    suffixes=("_egrid", "_calc"),
)

# plants with missing ba code
# ba_code_match[(ba_code_match['ba_code_calc'].isna()) & ~(ba_code_match['ba_code_egrid'].isna())]
ba_code_match[ba_code_match["ba_code_calc"] != ba_code_match["ba_code_egrid"]]

In [ ]:
# how many of these mismatches are for non-missing bas
ba_code_match[
    (ba_code_match["ba_code_calc"] != ba_code_match["ba_code_egrid"])
    & ~(ba_code_match["ba_code_egrid"].isna())
]

### Identify whether the fuel codes of each plant match

In [ ]:
fuel_match = egrid_plant.set_index("plant_id_eia")[
    ["plant_name", "plant_primary_fuel"]
].merge(
    annual_plant_results.set_index("plant_id_eia")[["plant_primary_fuel"]],
    how="inner",
    left_index=True,
    right_index=True,
    suffixes=("_egrid", "_calc"),
)

fuel_match[
    fuel_match["plant_primary_fuel_egrid"] != fuel_match["plant_primary_fuel_calc"]
]



# Split data into different groups based on known discrepencies

There are certain classes of plants where there are known issues that we know will prevent the totals from matching:
- There are certain power plants that are missing altogether from eGRID
- Geothermal power plants: EGRID sometimes has incorrect geotypes that may lead to different emissions estimates
- Nuclear power plants: eGRID is generally missing fuel consumption data
- Fuel cells are assumed by eGRID to have zero emissions
- Plants that only report CEMS data for part of the year are sometimes missing data for the rest of the year in eGRID
- CHP plants use a slightly different methodology
- plants that report data to the bf table or gen table in EIA-923 may be missing data from teh generation and fuel table.


We could identify plants where the total fuel doesn't match EIA-923, but matches the total fuel for a subset of prime movers

In [ ]:
pudl_out = load_data.initialize_pudl_out(year)
eia923_allocated = pd.read_csv(
    f"{outputs_folder()}{year}/eia923_allocated_{year}.csv",
    dtype=get_dtypes(),
    parse_dates=["report_date"],
)

annual_plant_results_segmented = validation.segment_plants_by_known_issues(annual_plant_results, egrid_plant, eia923_allocated, pudl_out, PLANTS_MISSING_FROM_EGRID)


## Plant Metric

In [ ]:
flag_columns = ['flag_missing_egrid', 'flag_geothermal','flag_nuclear', 'flag_fuel_cell', 'flag_partial_year', 'flag_chp', 'flag_bf_gen_reporter']

# all data without known issues
segment_to_compare = annual_plant_results_segmented[annual_plant_results_segmented[flag_columns].sum(axis=1) == 0]

comparison_count, compared = validation.compare_plant_level_results_to_egrid(
    segment_to_compare, egrid_plant, PLANTS_MISSING_FROM_EGRID
)
comparison_count


In [ ]:
validation.compare_egrid_fuel_total(segment_to_compare, egrid_plant).sum()


### Explore a specific set of plants

In [ ]:
# specify the dataframe, metric, and status to explore
comparison_df = compared
metric = "fuel_consumed_for_electricity_mmbtu"
status = "-50% to -10%"

# show the data
columns_to_show = ["plant_name","ba_code","state", metric, f"{metric}_status"]
comparison_df.loc[(comparison_df[f"{metric}_status"] == status), columns_to_show]

## Compare Annual BA values to eGRID BA file

In [ ]:
year = 2020
path_prefix = year

# Load the eGRID plant table
egrid_plant = validation.load_egrid_plant_file(year)

egrid_ba = validation.load_egrid_ba_file(year)

# aggregate the plant data up to the BA level
data_columns = [
    "net_generation_mwh",
    "fuel_consumed_mmbtu",
    "fuel_consumed_for_electricity_mmbtu",
    "co2_mass_lb",
    "co2_mass_lb_adjusted",
]
egrid_plant_ba_agg = egrid_plant.groupby(["ba_code"]).sum()[data_columns].reset_index()



In [ ]:
# load our annual ba data
DATA_COLUMNS = [
    "net_generation_mwh",
    "fuel_consumed_mmbtu",
    "fuel_consumed_for_electricity_mmbtu",
    "co2_mass_lb",
    "co2_mass_lb_adjusted",
]

calculated_ba = []

for filename in os.listdir(
    f"{results_folder()}{path_prefix}/power_sector_data/annual/us_units/"
):
    ba = filename.split(".")[0]
    ba_data = pd.read_csv(
        f"{results_folder()}{path_prefix}/power_sector_data/annual/us_units/{filename}",
        usecols=(["fuel_category"] + DATA_COLUMNS),
    )
    ba_data = ba_data[ba_data["fuel_category"] == "total"].drop(
        columns=["fuel_category"]
    )
    ba_data["ba_code"] = ba
    ba_data = ba_data[["ba_code"] + DATA_COLUMNS]
    calculated_ba.append(ba_data)

calculated_ba = pd.concat(calculated_ba, axis=0)


In [ ]:
percent_diff_from_egrid = (
    (
        calculated_ba.set_index("ba_code").replace(0, 0.1)
        - egrid_plant_ba_agg.set_index("ba_code").replace(0, 0.1)
    )
    / egrid_plant_ba_agg.set_index("ba_code").replace(0, 0.1)
).round(2)

In [ ]:
percent_diff_from_egrid.sort_values(by="net_generation_mwh")

In [ ]:
# divide our calculation by the BA totals from eGRID
# if there are 0 values, replace with 0.1, so that div by zero doesn't return missing value
ba_metric = (
    calculated_ba.replace(0, 0.1)
    .set_index("ba_code")
    .div(egrid_plant_ba_agg.set_index("ba_code").replace(0, 0.1))
    .sort_values(by="co2_mass_lb")
    .round(3)
)

total = pd.DataFrame(
    calculated_ba[data_columns]
    .sum()
    .div(egrid_plant_ba_agg[data_columns].sum())
    .rename("Total")
).T

# calculate the difference in the number of plants in each region
# plant_count = (plant_annual_total.groupby('ba_code', dropna=False).count()['plant_id_egrid'] - egrid_plant.groupby('ba_code', dropna=False).count()['plant_id_egrid']).rename('num_plants')
# ba_metric = ba_metric.merge(plant_count, how='left', left_index=True, right_index=True).drop(columns=['plant_id_egrid']).sort_index()

ba_metric = pd.concat([ba_metric, total], axis=0).round(2)

ba_metric = ba_metric[data_columns]

columns_to_check = [
    "net_generation_mwh",
    "fuel_consumed_mmbtu",
    "fuel_consumed_for_electricity_mmbtu",
    "co2_mass_lb",
]

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(ba_metric[~(ba_metric[columns_to_check] == 1).all(axis=1)])


## Explore specific plants

### Notes

BA Totals
 - TEPC and SRP are off because the Gila River Generator is shared between SRP and TEPC, and eGRID reports all generation from this project belonging to TEPC


In [ ]:
plant_to_explore = 58223


In [ ]:
egrid_plant[egrid_plant["plant_id_eia"] == plant_to_explore]

In [ ]:
annual_plant_results[annual_plant_results["plant_id_eia"] == plant_to_explore]

In [ ]:
eia923_allocated[eia923_allocated["plant_id_eia"] == plant_to_explore]

In [ ]:
eia923_allocated.loc[eia923_allocated["plant_id_eia"] == plant_to_explore, ["generator_id","subplant_id"]].drop_duplicates()